This notebook is used to organize the experiments. If you just want the best models found, you can refer to the `3_model_training` notebook.

In [1]:
import os

import mlflow
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, log_loss
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from functools import partial

os.sys.path.append(os.path.abspath('../src'))
from data import loader
from data import preprocessor

mapname = 'Town01'

In [2]:
def create_and_train_model(model, args):
    """
    Function to create and train a model with given parameters.

    Args:
        model: The model to be trained.
        args: Arguments for training the model.

    Returns:
        result: The result of the training process.
    """
    result = {}
    # Train model with current hyperparameters
    print(f"Params: {args}")
    md = model(**args)
    md.fit(X_train, y_train)
    # Predict on the validation set
    y_pred = md.predict(X_val)
    # Log training results
    result["f1_score"] = f1_score(y_val, y_pred, average='weighted')
    result["accuracy"] = accuracy_score(y_val, y_pred)
    result["recall"] = recall_score(y_val, y_pred, average='weighted')
    result["precision"] = precision_score(y_val, y_pred, average='weighted')
    result["model"] = md

    return result

In [3]:
def objective(params, model_name):
    """
    Objective function for hyperparameter optimization.
    This function will be called by Hyperopt for each trial.
    """
    with mlflow.start_run(nested=True, run_name=f"{model_name}_{params}"):
        # Log hyperparameters being tested
        mlflow.log_params(params)

        # Train model with current hyperparameters
        result = create_and_train_model(
            model,
            args=params
        )

        # Log training results
        mlflow.log_metrics(
            {
                "accuracy": result["accuracy"],
                "recall": result["recall"],
                "precision": result["precision"],
                "f1_score": result["f1_score"],
            }
        )

        # Log the trained model
        mlflow.sklearn.log_model(
            result["model"], name=model_name, signature=signature)

        # Return loss for Hyperopt (it minimizes)
        return {"loss": -result["f1_score"], "status": STATUS_OK, "model": f'{model_name}_{params}'}

In [4]:
def run_experiments(run_name, max_evals, search_space, data_name, features_names, model_name):
    # Create or set experiment

    print(
        f"This will run {max_evals} trials to find optimal hyperparameters...")

    with mlflow.start_run(run_name=run_name):

        mlflow.set_tag("model_type", model_name)

        # Log experiment metadata
        mlflow.log_params(
            {
                "optimization_method": "Tree-structured Parzen Estimator (TPE)",
                "max_evaluations": max_evals,
                "objective_metric": "log_loss",
                "dataset": data_name,
                "features": features_names,
            }
        )

        # Run optimization
        trials = Trials()
        objective_with_name = partial(objective, model_name=model_name)

        best_params = fmin(
            fn=objective_with_name,
            space=search_space,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials,
            verbose=True,
        )

        # Find and log best results
        best_trial = min(trials.results, key=lambda x: x["loss"])
        best_f1_score = -best_trial["loss"]

        # Log optimization results
        mlflow.log_params(best_params)
        mlflow.log_params({"best_model": best_trial["model"]})
        mlflow.log_metrics(
            {
                "best_f1_score": best_f1_score,
                "total_trials": len(trials.trials),
                "optimization_completed": 1,
            }
        )

    return best_f1_score 


In [5]:
client = mlflow.tracking.MlflowClient("http://127.0.0.1:8080")
mlflow.set_tracking_uri("http://127.0.0.1:8080")

## Preparing the Data

In [6]:
data_path = '../data'
uah_training = pd.read_csv(f'{data_path}/base/training_set_uah.csv')
uah_validation = pd.read_csv(f'{data_path}/base/validation_set_uah.csv')

carla_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_fixed.csv').drop(columns=['origin'])
carla_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_llm.csv').drop(columns=['origin'])

sumo_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_fixed.csv').drop(columns=['origin'])
sumo_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_llm.csv').drop(columns=['origin'])

carla_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_fixed.csv')
carla_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_llm.csv')

sumo_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_fixed.csv')
sumo_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_llm.csv')

Applying sliding windows to UAH, SUMO and CARLA full data

In [7]:
window_size = 10
step_size = 5
one_hot_keys = {
    'normal': 0,
    'aggressive': 1
}

Defining the search space for each model

In [8]:
max_evals = 5

search_space_rf = {
    "n_estimators": hp.choice("n_estimators", [20, 50, 100, 200, 500]),
    "max_depth": hp.choice("max_depth", [None, 5, 10, 20, 50]),
    "random_state": 42,
}
search_space_svc = {
    "C": hp.choice("C", [0.01, 0.05, 0.1, 0.5, 1]),  # Regularization parameter
    "kernel": hp.choice("kernel", ["rbf"]),  # Kernel type
    "gamma": hp.choice("gamma", [0.001, 0.01, 0.1, 1, 10]),  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
}
search_space_svc_linear = {
    "C": hp.choice("C", [10**-4, 10**-3, 10**-2, 0.1, 1])  # Regularization parameter
}
search_space_xgb = {
    "n_estimators": hp.choice("n_estimators", [200, 500, 700, 1000]),
    "max_depth": hp.choice("max_depth", [None, 5, 10]),
    "learning_rate": hp.choice("learning_rate", [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]),  # Learning rate
    # "subsample": hp.uniform("subsample", 0.5, 1.0),
    # "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
    "random_state": 42,
}

## Real Only

In [9]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH driveset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [10]:
columns_to_keep = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'speed', 'label']

X_train, y_train = preprocessor.sliding_windows(uah_training[columns_to_keep], window_size=window_size, step_size=step_size)
X_val, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH')
data_name = 'UAH'

In [11]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 20, 'n_estimators': 200, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/b4545e6d62fb46ee9f6327196b0e46fd

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824

Params: {'max_depth': 50, 'n_estimators': 20, 'random_state': 42}               
🏃 View run RandomForestClassifier_{'max_depth': 50, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/6b7d4a795d7b4e998301d39109dd01bc

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824   

Params: {'max_depth': 10, 'n_estimators': 200, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/cb94b8efb8bb4e7c

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.01, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/3547e21a18804c97a66c4ecb8742fb8a

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824  

Params: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}                                
 60%|██████    | 3/5 [02:52<02:10, 65.14s/trial, best loss: -0.750378634529194]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/3f6d065d3a084c8685ebc7d99dfeca1d

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824  

Params: {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}                               
 80%|████████  | 4/5 [04:39<01:21, 81.39s/trial, best loss: -0.750378634529194]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.01, 'gamma': 1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/1627897df74d4bfdbad38ca7485473af

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824  

100%|██████████| 5/5 [05:56<00:00, 71.30s/trial, best loss: -0.750378634529194]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/71e1a6a806be4bf3877addaf7602f185
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824
This will run 5 trials to find optimal hyperparameters...
Params: {'C': 0.1}                                   
🏃 View run LinearSVC_{'C': 0.1} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/961149dd405b4f318d2214cc3fcf69cb

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824

Params: {'C': 0.01}                                                             
🏃 View run LinearSVC_{'C': 0.01} at: http://127.0.0.1:8080/#/experiments/68041551005967282

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run XGBClassifier_{'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/b78074ff33a24aa6b3cc0cbcee31bd8d

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824

Params: {'learning_rate': 0.4, 'max_depth': None, 'n_estimators': 700, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.4, 'max_depth': None, 'n_estimators': 700, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/d4282d0f88fc427e937b8b21c6f41ea2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/680415510059672824   

Params: {'learning_rate': 0.5, 'max_depth': None, 'n_estimators': 1000, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.5, 'max_depth': None, 'n_estimators': 1000, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/680415510059672824/runs/9f292825a3fc4dfcbee22a3be6dd4269

🧪 View experiment at: http://127.0.0.1:

0.7218176596094258

## SUMO Only

In [12]:
# SUMO
X_sumo_fixed, y_sumo_fixed = preprocessor.sliding_windows(sumo_fixed, window_size=window_size, step_size=step_size)
X_sumo_llm, y_sumo_llm = preprocessor.sliding_windows(sumo_llm, window_size=window_size, step_size=step_size)

In [13]:
# Getting the validation for SUMO variables
columns_to_keep = sumo_fixed.columns.tolist()
print("Columns to keep for validation:", columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc', 'angle', 'speed', 'gyro_z', 'label']


### Fixed

In [14]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the SUMO Fixed dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_SUMO_Fixed", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [15]:
X_train, y_train = X_sumo_fixed, y_sumo_fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_SUMO_Fixed')
data_name = 'SUMO_Fixed'

In [16]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 20, 'n_estimators': 200, 'random_state': 42}
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/8e66cf6b774a42fabba32cdf451ca385

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339

Params: {'max_depth': None, 'n_estimators': 100, 'random_state': 42}             
 20%|██        | 1/5 [00:27<01:48, 27.09s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 100, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/a286d0e07667496a83c8b98124ac2e23

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339    

Params: {'max_depth': 20, 'n_estimators': 20, 'random_state': 42}                
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/52e6b182aa3949d6882a2b43a7186909

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339    

Params: {'max_depth': 5, 'n_estimators': 20, 'random_state': 42}                 
🏃 View run RandomForestClassifier_{'max_depth': 5, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/66b02c49efce4fbeab97da377abec8ee

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339   

Params: {'max_depth

...

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/c3a723a8dfe2439699f79bd7477637d2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339    

Params: {'C': 0.5, 'gamma': 0.01, 'kernel': 'rbf'}                                
 80%|████████  | 4/5 [06:19<01:49, 109.46s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.5, 'gamma': 0.01, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/70e80a21dacc4d89b0e55594d9e8c773

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339     

100%|██████████| 5/5 [07:17<00:00, 87.51s/trial, best loss: -0.33310749162724596] 
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/87b6ac0425544845b29b3a001647d7b8
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339
This will run 5 trials to find optimal hyperparameters...
Params: {'C': 0.01}                                  
🏃 View run LinearSVC_{'C': 0.01} at: http://127.0.0.1:8080/#/experiments/319021118004357339/runs/97867ad14d8f45eb86c710a26dc947e5

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/319021118004357339

Params: {'C': 0.0001}                                                            
🏃 View run LinearSVC_{'C': 0.0001} at: http://127.0.0.1:8080/#/experiments/31902

0.33730962743803933

### LLM

In [17]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the SUMO LLM dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_SUMO_LLM", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [18]:
X_train, y_train = X_sumo_llm, y_sumo_llm

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

mlflow.set_experiment('Driver_Behavior_Models_SUMO_LLM')
data_name = 'SUMO_LLM'

In [19]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': None, 'n_estimators': 200, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/443228b956a147779a5df4cf5dd33e9e

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287

Params: {'max_depth': None, 'n_estimators': 200, 'random_state': 42}            
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/8187a5c1de6742cc83eb1a9ae59b591e

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287   

Params: {'max_depth': 20, 'n_estimators': 20, 'random_state': 42}               
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/1f0fac16f5

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/255616c0636b44a5883381af59b29860

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287

Params: {'C': 0.05, 'gamma': 1, 'kernel': 'rbf'}                                  
 20%|██        | 1/5 [02:59<11:57, 179.41s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.05, 'gamma': 1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/d491031fad2844df9139a2c869d26abb

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287     

Params: {'C': 0.5, 'gamma': 0.001, 'kernel': 'rbf'}                               
🏃 View run SVC_{'C': 0.5, 'gamma': 0.001, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/7ad60986d6b84a96ae8821a6e4e00eef

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287     

Params: {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}                                 
 60%|██████    | 3/5 [06:13<03:47, 113.72s/trial, best loss: -0.33641148675357896]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.01, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/1618621b98474504b12406a651b0a05b

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287     

Params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}                               
 80%|████████  | 4/5 [07:52<01:48, 108.09s/trial, best loss: -0.33641148675357896]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/bb51dc4f915348f381f1fe2aa98ef422

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287     

100%|██████████| 5/5 [09:18<00:00, 111.78s/trial, best loss: -0.33641148675357896]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/f3ae6543bd6140d19a6b95513c05d58b
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287
This will run 5 trials to find optimal hyperparameters...
Params: {'C': 1}                                     
🏃 View run LinearSVC_{'C': 1} at: http://127.0.0.1:8080/#/experiments/945333525365379287/runs/d8bd2b2b37334864b29f832ba5c8ea17

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/945333525365379287

Params: {'C': 1}                                                                
🏃 View run LinearSVC_{'C': 1} at: http://127.0.0.1:8080/#/experiments/9453335253653

0.5748710014522482

## Carla Only

In [20]:
X_carla_fixed, y_carla_fixed = preprocessor.sliding_windows(carla_fixed, window_size=window_size, step_size=step_size)
X_carla_llm, y_carla_llm = preprocessor.sliding_windows(carla_llm, window_size=window_size, step_size=step_size)

In [21]:
# Getting the validation for SUMO variables
columns_to_keep = carla_fixed.columns.tolist()
print("Columns to keep for validation:", columns_to_keep)
X_val_base, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'label']


### Fixed

In [22]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the CARLA Fixed dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_CARLA_Fixed", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [23]:
X_train, y_train = X_carla_fixed, y_carla_fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

mlflow.set_experiment('Driver_Behavior_Models_CARLA_Fixed')
data_name = 'CARLA_Fixed'

In [24]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 10, 'n_estimators': 20, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/4c7a3ef88d564a37bc32a0ec6c7d9ba2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225

Params: {'max_depth': 10, 'n_estimators': 200, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/c2dc35308fa342bf922068f3e51682f8

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225   

Params: {'max_depth': None, 'n_estimators': 200, 'random_state': 42}            
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/265818ea574e433

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/b5fc43555b77453fbcb123bec110377d

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225

Params: {'C': 1, 'gamma': 10, 'kernel': 'rbf'}                                   
 20%|██        | 1/5 [00:19<01:19, 19.97s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/b073b5f22f044eacb182960492692aaa

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225    

Params: {'C': 0.05, 'gamma': 0.1, 'kernel': 'rbf'}                               
 40%|████      | 2/5 [01:23<02:16, 45.57s/trial, best loss: -0.33310749162724596]

...

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run LinearSVC_{'C': 1} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/e72fc50c685e483b9f6474577af0a5b2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225    

Params: {'C': 0.0001}                                                            
 80%|████████  | 4/5 [00:07<00:01,  1.93s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run LinearSVC_{'C': 0.0001} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/f4fec242e5ac44a0ac72099eed1bba26

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225    

100%|██████████| 5/5 [00:09<00:00,  1.83s/trial, best loss: -0.33310749162724596]
🏃 View run svc-linear-sweep at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/7e047b45c06a42838ec9c54faff142ae
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225
This will run 5 trials to find optimal hyperparameters...
Params: {'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 500, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/219343887402467225/runs/4542ba0acd2445729d9e03a293171577

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/219343887402467225

Params: {'learning_rate': 0.4, 'max_depth': 10, 'n_estimators': 700, 'ra

0.3934144677894773

### LLM

In [25]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the CARLA LLM dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_CARLA_LLM", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [26]:
X_train, y_train = X_carla_llm, y_carla_llm

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

mlflow.set_experiment('Driver_Behavior_Models_CARLA_LLM')
data_name = 'CARLA_LLM'

In [27]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': None, 'n_estimators': 50, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 50, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/2300379d66334c93881158be9f3ea81f

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628

Params: {'max_depth': None, 'n_estimators': 500, 'random_state': 42}             
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/095d9ff58f5f48079cc10a6e737e8a83

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628    

Params: {'max_depth': 10, 'n_estimators': 500, 'random_state': 42}               
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/251fc936

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/64a44e1539384e6d820f902973d96d4a

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628

Params: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}                              
 20%|██        | 1/5 [01:25<05:42, 85.73s/trial, best loss: -0.33310749162724596]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/dd8a74273b5f4533acdbbe006d209675

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628    

Params: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}                             
 40%|████      | 2/5 [01:57<02:41, 53.73s/trial, best loss: -0.33310749162724596]

...

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run LinearSVC_{'C': 0.1} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/aa25e87e2cea499d81663dc59fe736b6

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628    

100%|██████████| 5/5 [00:09<00:00,  1.91s/trial, best loss: -0.33310749162724596]
🏃 View run svc-linear-sweep at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/1fc740d0a526463192b33156dfb013c4
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628
This will run 5 trials to find optimal hyperparameters...
Params: {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 500, 'random_state': 42}
🏃 View run XGBClassifier_{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/a7ed20d649c8410d930475d2d9557bb6

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628

Params: {'learning_rate': 0.4, 'max_depth': 5, 'n_estimators': 700, 'random_s

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run XGBClassifier_{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 700, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/8ec42b08cdb44a51b7b693504e4d6ecd

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628    

100%|██████████| 5/5 [00:37<00:00,  7.55s/trial, best loss: -0.40131969784645705]
🏃 View run xgb-sweep at: http://127.0.0.1:8080/#/experiments/339489166525188628/runs/15ee4321853346d89320159e72d27843
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/339489166525188628


0.40131969784645705

## Real + SUMO (20%)

In [28]:
sumo_uah_fixed_20 = preprocessor.fill_synthetic_data(sumo_uah_fixed, 0.2)
sumo_uah_llm_20 = preprocessor.fill_synthetic_data(sumo_uah_llm, 0.2)

In [29]:
# SUMO_UAH
X_sumo_uah_fixed_20, y_sumo_uah_fixed_20 = preprocessor.sliding_windows(sumo_uah_fixed_20, window_size=window_size, step_size=step_size)
X_sumo_uah_llm_20, y_sumo_uah_llm_20 = preprocessor.sliding_windows(sumo_uah_llm_20, window_size=window_size, step_size=step_size)

In [30]:
# Getting the validation for SUMO variables
columns_to_keep = sumo_uah_fixed_20.columns.tolist()
print("Columns to keep for validation:", columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc', 'angle', 'speed', 'gyro_z', 'label']


### Fixed

In [31]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH + SUMO Fixed (20%) dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH_SUMO_Fixed_20", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [32]:
X_train, y_train = X_sumo_uah_fixed_20, y_sumo_uah_fixed_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH_SUMO_Fixed_20')
data_name = 'UAH_SUMO_Fixed_20'

In [33]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 10, 'n_estimators': 200, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/2a97cc758d57472fb6e6357fae49a854

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/466256615552701962

Params: {'max_depth': 50, 'n_estimators': 200, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 50, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/0c075ebb9bc74c509cb15e5f9d896fab

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/466256615552701962   

Params: {'max_depth': None, 'n_estimators': 200, 'random_state': 42}            
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/9f777a2913904

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.05, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/1338367f8f194a98b97976a15b2fc42e

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/466256615552701962   

100%|██████████| 5/5 [02:26<00:00, 29.38s/trial, best loss: -0.7087121973722031]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/45a49c808abc41feaf16d913c16461ea
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/466256615552701962
This will run 5 trials to find optimal hyperparameters...
Params: {'C': 0.0001}                                
🏃 View run LinearSVC_{'C': 0.0001} at: http://127.0.0.1:8080/#/experiments/466256615552701962/runs/9c94c73f5dc5453286e8411ffb692faa

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/466256615552701962

Params: {'C': 1}                                                                
🏃 View run LinearSVC_{'C': 1} at: http://127.0.0.1:8080/#/experiments/46625661555270

0.7027595665857383

### LLM

In [34]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH + SUMO LLM (20%) dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH_SUMO_LLM_20", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [35]:
X_train, y_train = X_sumo_uah_llm_20, y_sumo_uah_llm_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH_SUMO_LLM_20')
data_name = 'UAH_SUMO_LLM_20'

In [36]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 5, 'n_estimators': 200, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 5, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/870626999030178392/runs/8e108fd2ed4a4501a23f0bf4493c1e7d

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/870626999030178392

Params: {'max_depth': 20, 'n_estimators': 200, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 200, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/870626999030178392/runs/22fb2e0d893345b8b6637b0185c8a647

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/870626999030178392   

Params: {'max_depth': 20, 'n_estimators': 500, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/870626999030178392/runs/d707626ddee94fd59

0.6810211254531792

## Real + CARLA (20%)

In [37]:
carla_uah_fixed_20 = preprocessor.fill_synthetic_data(carla_uah_fixed, 0.2)
carla_uah_llm_20 = preprocessor.fill_synthetic_data(carla_uah_llm, 0.2)

In [38]:
# CARLA_UAH
X_carla_uah_fixed_20, y_carla_uah_fixed_20 = preprocessor.sliding_windows(carla_uah_fixed_20, window_size=window_size, step_size=step_size)
X_carla_uah_llm_20, y_carla_uah_llm_20 = preprocessor.sliding_windows(carla_uah_llm_20, window_size=window_size, step_size=step_size)

In [39]:
# Getting the validation for SUMO variables
columns_to_keep = carla_uah_fixed_20.columns.tolist()
print("Columns to keep for validation:", columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'label']


### Fixed

In [40]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH + CARLA Fixed (20%) dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH_CARLA_Fixed_20", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [41]:
X_train, y_train = X_carla_uah_fixed_20, y_carla_uah_fixed_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH_CARLA_Fixed_20')
data_name = 'UAH_CARLA_Fixed_20'

In [42]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 5, 'n_estimators': 50, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 5, 'n_estimators': 50, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/010615483d9a4396b3ee6f39862e0bb2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/435216938818414363

Params: {'max_depth': None, 'n_estimators': 100, 'random_state': 42}            
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 100, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/31c7307f34dd42748569b72cba2cbb00

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/435216938818414363   

Params: {'max_depth': 20, 'n_estimators': 500, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 500, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/85212756f3ec4777a

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 1, 'gamma': 1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/946db53cd94c41e89dab7ecefdea9833

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/435216938818414363   

Params: {'C': 0.5, 'gamma': 0.01, 'kernel': 'rbf'}                              
🏃 View run SVC_{'C': 0.5, 'gamma': 0.01, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/657907ce6c024a3a987031fb6a6e4f5a

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/435216938818414363   

Params: {'C': 0.05, 'gamma': 0.001, 'kernel': 'rbf'}                            
🏃 View run SVC_{'C': 0.05, 'gamma': 0.001, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/435216938818414363/runs/77ef50864f4443ce81afd611a221bc03

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/435216938818414363   

100%|██████████| 5/5 [04:52<00:00, 58.51s/trial, best loss: -0.5980641760820076]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/

0.6054982553607615

### LLM

In [43]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Experiment to train models on the UAH + CARLA LLM (20%) dataset."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "driver-behavior-prediction",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
try:
    driver_behavior_experiment = client.create_experiment(
        name="Driver_Behavior_Models_UAH_CARLA_LLM_20", tags=experiment_tags
    )
except mlflow.exceptions.RestException as e:
    print(f"Experiment already exists: {e}")

In [44]:
X_train, y_train = X_carla_uah_llm_20, y_carla_uah_llm_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

signature = infer_signature(X_train, y_train)
mlflow.set_experiment('Driver_Behavior_Models_UAH_CARLA_LLM_20')
data_name = 'UAH_CARLA_LLM_20'

In [45]:
model = RandomForestClassifier
run_experiments('rf-sweep', max_evals=max_evals, search_space=search_space_rf, data_name=data_name, model_name = 'RandomForestClassifier', features_names=columns_to_keep)
model = SVC
run_experiments('svc-sweep', max_evals=max_evals, search_space=search_space_svc, data_name=data_name, model_name = 'SVC', features_names=columns_to_keep)
model = LinearSVC
run_experiments('svc-linear-sweep', max_evals=max_evals, search_space=search_space_svc_linear, data_name=data_name, model_name = 'LinearSVC', features_names=columns_to_keep)
model = xgb.XGBClassifier
run_experiments('xgb-sweep', max_evals=max_evals, search_space=search_space_xgb, data_name=data_name, model_name = 'XGBClassifier', features_names=columns_to_keep)

This will run 5 trials to find optimal hyperparameters...
Params: {'max_depth': 10, 'n_estimators': 20, 'random_state': 42}
🏃 View run RandomForestClassifier_{'max_depth': 10, 'n_estimators': 20, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/88f25de0bf2345a9925d87db752a1a3f

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444

Params: {'max_depth': 20, 'n_estimators': 50, 'random_state': 42}              
🏃 View run RandomForestClassifier_{'max_depth': 20, 'n_estimators': 50, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/bcc200c724d1418db667c886669e62a6

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444  

Params: {'max_depth': None, 'n_estimators': 100, 'random_state': 42}            
🏃 View run RandomForestClassifier_{'max_depth': None, 'n_estimators': 100, 'random_state': 42} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/d07a145b00e94bb780

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.01, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/4ecdee0fea9542d3a8d2ead106f39fd5

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444   

Params: {'C': 0.05, 'gamma': 0.1, 'kernel': 'rbf'}                              
 40%|████      | 2/5 [02:00<03:17, 65.68s/trial, best loss: -0.5202081260565236]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.05, 'gamma': 0.1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/372648dc49254e5b8ac5ce74c24f7b28

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444   

Params: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}                                 
🏃 View run SVC_{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/19b8dafc4b59489cb2f9a737d49f1912

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444   

Params: {'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}                                
 80%|████████  | 4/5 [04:04<01:06, 66.25s/trial, best loss: -0.5980641760820076]

/home/renan/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🏃 View run SVC_{'C': 0.1, 'gamma': 10, 'kernel': 'rbf'} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/f5172ee81889471d8ab0445114d2697f

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444   

100%|██████████| 5/5 [05:54<00:00, 70.90s/trial, best loss: -0.5980641760820076]
🏃 View run svc-sweep at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/c27c775ce5fe475e970d94e5c28e5982
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444
This will run 5 trials to find optimal hyperparameters...
Params: {'C': 0.1}                                   
🏃 View run LinearSVC_{'C': 0.1} at: http://127.0.0.1:8080/#/experiments/520029334633823444/runs/a76618065b9e4e228dfd1bc951a86ab4

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/520029334633823444

Params: {'C': 0.1}                                                               
🏃 View run LinearSVC_{'C': 0.1} at: http://127.0.0.1:8080/#/experiments/520029334633823

0.6714134517080457